In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
import kagglehub

In [ ]:


# Download latest version
path = kagglehub.dataset_download("govindaramsriram/energy-consumption-dataset-linear-regression")

print("Path to dataset files:", path)

In [ ]:
test_data = pd.read_csv(path+'//test_energy_data.csv')
train_data = pd.read_csv(path+'//train_energy_data.csv')